# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import glob
import yaml
import subprocess

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Set3')

import IPython.display as display
from IPython.display import Video

from PIL import Image
import cv2

import torch

from ultralytics import YOLO

In [2]:
import ultralytics
print(ultralytics.__version__)

8.1.29


# CFG

In [ ]:
class CFG:
    ### inference: use any pretrained or custom model
    # WEIGHTS = 'yolov8x.pt' # yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
    WEIGHTS = 'models/best.pt'

    CONFIDENCE = 0.60
    CONFIDENCE_INT = int( round(CONFIDENCE * 100, 0) )

    with open('datasets/annotated_dataset/classes.txt', 'r') as file:
        CLASSES_TO_DETECT = list(range(len(file.readlines())))

    VERTICES_POLYGON = np.array([[200,720], [0,700], [500,620], [990,690], [820,720]])

    EXP_NAME = 'ppe'

    ### just some video examples
    VID_001 = 'test_videos/IMG_4504.MOV'

    ### choose filepath to make inference on (image or video)
    PATH_TO_INFER_ON = VID_001
    EXT = PATH_TO_INFER_ON.split('.')[-1] # get file extension
    FILENAME_TO_INFER_ON = PATH_TO_INFER_ON.split('/')[-1].split('.')[0] # get filename

    ### paths
    ROOT_DIR = 'test_videos'
    OUTPUT_DIR = './'

In [7]:
glob.glob(CFG.ROOT_DIR + '*')

['test_videos']

## Image utils

In [8]:
def get_image_properties(image):
    if isinstance(image, str):
        # If image is a file path, read the image
        img = cv2.imread(image)
        if img is None:
            raise ValueError("Could not read image file")
    elif isinstance(image, np.ndarray):
        # If image is already a NumPy array, use it directly
        img = image
    else:
        raise ValueError("Input must be a file path or a NumPy array")

    # Get image properties
    properties = {
        "width": img.shape[1],
        "height": img.shape[0],
        "channels": img.shape[2] if len(img.shape) == 3 else 1,
        "dtype": img.dtype,
    }

    return properties

In [9]:
def display_image(image, print_info = True, hide_axis = False):
    if isinstance(image, str):  # Check if it's a file path
        img = Image.open(image)
        plt.imshow(img)
    elif isinstance(image, np.ndarray):  # Check if it's a NumPy array
        image = image[..., ::-1]  # BGR to RGB
        img = Image.fromarray(image)
        plt.imshow(img);
    else:
        raise ValueError("Unsupported image format")

    if print_info:
        print('Type: ', type(img), '\n')
        print('Shape: ', np.array(img).shape, '\n')

    if hide_axis:
        plt.axis('off')

    plt.show()

## Video utils

In [10]:
def get_video_properties(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        raise ValueError("Could not open video file")

    # Get video properties
    properties = {
        "fps": int(cap.get(cv2.CAP_PROP_FPS)),
        "frame_count": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        "duration_seconds": int( cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS) ),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        "codec": int(cap.get(cv2.CAP_PROP_FOURCC)),
    }

    # Release the video capture object
    cap.release()

    return properties

In [11]:
### testing function
video_properties = get_video_properties(CFG.VID_001)
video_properties

{'fps': 30,
 'frame_count': 216,
 'duration_seconds': 7,
 'width': 464,
 'height': 848,
 'codec': 875967080}

In [14]:
model = YOLO(CFG.WEIGHTS)

In [16]:
if torch.cuda.is_available():
    model.to('cuda:0')
else:
    print("CUDA device is not available. Running on CPU.")

CUDA device is not available. Running on CPU.


In [17]:
print('Device: ', model.device)
print('Weights: ', CFG.WEIGHTS)

Device:  cpu
Weights:  best.pt


In [18]:
# model.model
model.names

{0: 'Helmet',
 1: 'Safety_Goggles',
 2: 'Vest',
 3: 'Hairnet',
 4: 'Earplug',
 5: 'No_Helmet',
 6: 'No_Safety_Goggles',
 7: 'No_Vest',
 8: 'No_Hairnet',
 9: 'No_Earplug',
 10: 'Person'}

# ⚡Inference


In [23]:
%%time

results = model.predict(
    source = CFG.PATH_TO_INFER_ON,
    save = True,
    classes = CFG.CLASSES_TO_DETECT,
    conf = CFG.CONFIDENCE,
    save_txt = False,
    save_conf = False,
    show = True,
    # stream = True,
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/216) /Users/akbaig/Projects/Hackathon/PPE-Detection-YOLO/test_videos/IMG_4504.MOV: 640x352 (no detections), 88.0ms
video 1/1 (frame 2/216) /Users/akbaig/Projects/Hackathon/PPE-Detection-YOLO/test_videos/IMG_4504.MOV: 640x352 (no detections), 71.1ms
video 1/1 (frame 3/216) /Users/akbaig/Projects/Hackathon/PPE-Detection-YOLO/test_videos/IMG_4504.MOV: 640x352 1 Person, 72.8ms
video 1/1 (frame 4/216) /Users/akbaig/Projects/Hackathon/PPE-

In [24]:
OUT_VIDEO_NAME = 'runs/detect/predict2/{}.mp4'.format(CFG.FILENAME_TO_INFER_ON)
Video(data=OUT_VIDEO_NAME, embed=True, height=int(video_properties['height'] * 0.5), width=int(video_properties['width'] * 0.5))